# Loads
Topfarm supports load constraints by evaluating a trained surrogate model at each iteration. There are currently two different high-level load workflows implemented, namely the Dynamic Wake Meandering (DWM) and the Frandsen approach.
### DWM
In DWM the loads are calculated for all turbines for all flowcases and for all turbine interactions. This means that for each turbine it will look at the wakes coming from all other turbines (individually). The results are subsequently reduced to only one load per turbine by applying a soft max. This is done to create a differentiable expression for the max load instead of just taking the max load, which would not work well in a gradient-based optimization.
### Frandsen
In the Frandsen implementation, the turbulence is aggregated from different sectors with the Wöhler exponent as described in the IEC. Loads are then calculated based on the effective turbulence. Alternatively, if one would like to improve the fidelity by trading off some memory consumption the loads could be calculated based on the all wind direction sectors and subsequently aggregated with the Wöhler exponent.

## Load Surrogates
Topfarm can utilize surrogates trained with a range of different algorithms and softwares, namely scikit-learn, OpenTURNS, TensorFlow and the artificial neural networks code (wind2loads ANN) developed at DTU. 

## Install wind2loads
In this exercisw we will use the wind2loads artificial neural networks code to predict the turbine loads of the DTU 10MW reference turbine.

In [1]:
%%capture
try:
    import w2l
except:
    !pip install --upgrade git+https://gitlab.windenergy.dtu.dk/TOPFARM/wind2loads.git    
try:
    import workshop
except:
    try:
      !git clone https://gitlab.windenergy.dtu.dk/TOPFARM/workshop-december-2019.git
    except:
        pass
    import sys
    !pip install -e ./workshop-december-2019
    sys.path.append('./workshop-december-2019')
try:
    import topfarm
except:
    !pip install topfarm

## Import dependencies

In [2]:
import numpy as np
import topfarm
from topfarm import TopFarmProblem
from topfarm.easy_drivers import EasyScipyOptimizeDriver
from topfarm.constraint_components.spacing import SpacingConstraint
from topfarm.constraint_components.boundary import XYBoundaryConstraint
from topfarm.plotting import XYPlotComp

In [3]:
import workshop
import os
workshop.w2l_path = 'workshop-december-2019/workshop/wind2loads_ann'
import workshop.workflow as wf

IndexError: list index out of range

## Set up the problem
In this example we will import most of the code for setting up the workflow.

In [ ]:
def get_problem(maxiter=10, allowable_load=1.0, tol=1e-2):
    problem = TopFarmProblem(
            design_vars={topfarm.x_key: wf.site.initial_position[:, 0],
                         topfarm.y_key: wf.site.initial_position[:, 1]},
            cost_comp=wf.get_load_cost_comp(),
            driver=EasyScipyOptimizeDriver(maxiter=maxiter, tol=tol),
            constraints=[SpacingConstraint(wf.min_spacing),
                         XYBoundaryConstraint(wf.boundary),],
                post_constraints=[(ls, allowable_load) for ls in wf.load_signals],
            plot_comp=XYPlotComp(),
            approx_totals={'step':10},
            expected_cost=1e-6,)
    cost, state = problem.evaluate()
    max_load = {ls: np.array([problem[ls+'_abs'].max()]) for ls in wf.load_signals}
    problem.cost_comp.analytical_group.lifetime_comp.options['max_load'] = max_load
    return problem

## Run the optimization
In this example there is the option to change the maximum number of iterations, the tolerance of the optimization as well as the maximum allowable load. The maximum allowable load is set as a percentage of the loads calculated for the initial layout, meaning e.g. that allowable_load=1.0 is 100% of the initial loads. We will optimize on AEP of the wind farm.

In [ ]:
problem = get_problem(5, 1.0)
cost, state, recorder = problem.optimize(disp=True)

## Post Process
During the optimization all variable are recorded. After it is finished one can see all the recorded variables by typing `recorder.keys()` and plot them by writing `plt.plot([your variable],'.')`. We can use this procedure to examine how the load constraint has developed during the optimization:

First we show the relative loads meaning the ratio between the load at each iteration and the initial loads:

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
for ls in wf.load_signals:
    plt.plot(np.max(recorder[ls], axis=1),'.',label=ls)
plt.grid()
plt.title('Load constraints')
plt.legend()
plt.ylabel('Load / Initial Load')
plt.xlabel('iteration')
plt.show()

Secondly we can also plot the absolute magnitude of the loads as a function of the iterations:

In [ ]:
plt.figure()
for ls in wf.load_signals:
    ls += '_abs'
    plt.plot(np.max(recorder[ls], axis=1),'.',label=ls)
plt.grid()
plt.title('Load constraints')
plt.legend()
plt.ylabel('Load [kNm]')
plt.xlabel('iteration')
plt.show()